In [209]:
import pandas as pd
xls=pd.ExcelFile('Paavan Dairy-automated-invoice.xlsx')
client_details=pd.read_excel(xls,'Clients')
product_details=pd.read_excel(xls,'Products')
Bill=pd.read_excel(xls,'Sheet1')

In [210]:
final_bill={}
flag=0
for index,row in Bill.iterrows():
    client_id=row['Client Number']
    if client_id=='nan':
        break
    product_id=row['Product']
    quantity=row['Quantity']
    name=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Name'])[1:]).strip()
    name=(name.split('\n'))[0]
    street=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Street'])[1:]).strip()
    street=(street.split('\n'))[0]
    city=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'City'])[1:]).strip()
    city=(city.split('\n'))[0]
    Phone_Number=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Phone Num'])[1:]).strip()
    Phone_Number=(Phone_Number.split('\n'))[0]
    Product=(str(product_details.loc[product_details['Product nr']==row['Product'],'Description'])[1:]).strip()
    Product=(Product.split('\n'))[0]
    Price=(str(product_details.loc[product_details['Product nr']==row['Product'],'Price'])[1:]).strip()
    Price=(Price.split('\n'))[0]
    if row['Client Number'] not in final_bill.keys():
        prod={}
        prod[0]=({'product_id':product_id,'Product':Product,'Price':Price,'Quantity':quantity})
        final_bill[row['Client Number']]={'Client_Name':name,'Street':street,'City':city,'Phone_Number':Phone_Number,'prod':prod}
    else:
        prod={}
        prod=final_bill[row['Client Number']]['prod']
        product_number=prod.keys()
        count=len(product_number)
        for j in range(count):
            if prod[j]['product_id']==product_id:
                final_bill[row['Client Number']]['prod'][j]['Quantity']+=quantity
                flag=1
                break
        #print(name+str("")+str(flag))
        if flag==1:
            flag=0
        else:
            prod[count]=({'product_id':product_id,'Product':Product,'Price':Price,'Quantity':quantity})
            final_bill[row['Client Number']]={'Client_Name':name,'Street':street,'City':city,'Phone_Number':Phone_Number,'prod':prod}
        

In [211]:
df=(pd.DataFrame(final_bill))

In [212]:
df=df.transpose()

In [213]:
for index,j in df.iterrows():
    j['prod']=(pd.DataFrame(j['prod']))

In [214]:
df=df.transpose()

In [215]:
df[501]['prod'][0]

Price             50
Product       Bottle
Quantity           0
product_id      1002
Name: 0, dtype: object

In [232]:
import json
import random
import datetime
import pandas as pd
from pprint import pprint
file=open(r'C:\Users\Perfect Disk\Desktop\Bill Generator\rajat_bill.html','r')
file1=open(r'C:\Users\Perfect Disk\Desktop\Bill Generator\bill_gen2.html','w')
content=file.read()

In [233]:
content=content.replace('\t',"")
content = content.replace("\n","")

In [234]:
prototype=""
to ="to :"
findto=content.find(to)
prototype+=content[0:findto+len(to)]+" "+"Rajat Jain"

date = "Date :"
finddate=content.find(date)
now = datetime.datetime.now()
prototype+=content[findto+len(to):finddate+len(date)]+" "+str(now.date())

month ="Month :"
findmonth=content.find(month)
prototype+=content[finddate+len(date):findmonth+len(month)]+" "+str(now.strftime("%B"))

invoicenr = "Invoice nr :"
findinvoicenr = content.find(invoicenr)
prototype+=content[findmonth+len(month):findinvoicenr+len(invoicenr)]+" "+str(random.randint(1,100))

clientnr ="Client nr :"
findclientnr=content.find(clientnr)
prototype+=content[findinvoicenr+len(invoicenr):findclientnr+len(clientnr)]+" "+"501"

tbody="tbody"
findtbody=content.find(tbody)
prototype+=content[findclientnr+len(clientnr):findtbody+len(tbody)]

file1.write(prototype)
file1.close()